# Copy of Roberts approach in python to compute best emitting spo vs neutron output imaging time

- location of the MATLAB files: /Users/hkromer/02_PhD/02_Data/10.OutputVSEmittingSpot_Robert/

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Boundary conditions

- clarify the following with Robert:
    - units of FOV?
    - units of detector size?
    - what is the flux utilization?
    - imaging time raw?
    - imaging time effective>?
    - how does the imaging time have units seconds? Because of the magnification?

In [192]:
# model assumptions
d_so_min = 50 # source object minimum distance, mm
d_sd_min = 50 # source detector minimum distance, mm
d_od_min = 100 # object detector minimum distance, mm
d_sd_max = 10000 # source detector maximum distance, mm
d_so_max = 10000 # source object maximum distance, mm
FOV_min = 200 # field of view (degree) required to be satisfied
det_size_max = 1000 # maximum detector size, mm
B_d = 0.8 # assumed detector resolution, mm
resolution_goal = 1.2 # target overall maximum allowed resolution, mm
detector_efficiency = 0.05 # assumed detector efficiency, [-]
detector_counts = 3000 # required flat field counts per pixel for an image, [-]
output_bias = 2 # forward bias of output, for DD reaction, approx. vs. isotropic

## 2. Range of parameters to calculate for

In [193]:
# range of parameters to calculate for
ni = 100 # number of intervals to sample in emitting spot and source strength

f_min = 0.5 # lower limit of emitting spot to use, mm
f_max = 26 # upper limit of emitting spot to use, mm
f_list = np.linspace(f_min, f_max, ni) # focal spot, mm, over a range of f_min to f_max

e_min = 7.5 # lower limit of neutron output 1eX n/s
e_max = 12.5 # upper limit of neutron output 1eX n/s
exp_list = np.linspace(e_min, e_max, ni) # use outputs of e_min to e_max in steps of ni
o_list = np.asarray([10**e for e in exp_list]) # calculates output (neutrons/s) based on given exponent

## 3. Function to get the optimum geometry

In [ ]:
def get_optimum_geometry(
    output, # calculates output (neutrons/s) based on given exponent (this is o_list)
    f, # focal spot, mm
    output_bias, # forward bias of output, for DD reaction, approx. vs. isotropic
    d_so_min, # source object minimum distance, mm
    d_od_min, # object detector minimum distance, mm
    d_sd_max, # source detector maximum distance, mm
    d_so_max, # source object maximum distance, mm
    FOV_min, # field of view required to be satisfied
    det_size_max, # maximum detector size
    B_d, # assumed detector resolution, mm
    resolution_goal, # target overall maximum allowed resolution, mm
    detector_efficiency, # assumed detector efficiency
    detector_counts):# required flat field counts per pixel for an image, [-]
    
    # possible options for source to detector (d_sd) and source to object (d_so) distances
    ni = 100 # number of options in each d_sd and d_so
    d_so_s = pd.Series(np.arange(d_so_min,d_so_max, ni), name='d_so')
    d_sd_s = pd.Series(np.arange(d_sd_min,d_sd_max, ni), name='d_sd')
    df_img_times = pd.DataFrame(index=d_so_s, columns=d_sd_s) # rows: source to object; cols: source to detector distance
    
    # calculate values (ignoring boundary conditions) for each d_so and d_sd
    # magnification
    
    M_mat = d_sd_mat./d_so_mat;
    B_do_mat = B_d./M_mat;
    B_fo_mat = f*(1-1./M_mat);
    B_o_mat = sqrt(B_do_mat.^2+B_fo_mat.^2);
    flux_mat = output/4/pi./(d_sd_mat.^2)*output_bias;
    flux_utilization_mat = M_mat.^2;
    imaging_time_raw_mat = detector_counts./flux_mat/detector_efficiency;
    imaging_time_effective_mat = imaging_time_raw_mat.*flux_utilization_mat;

In [38]:
df_distances.head()

d_sd,100,200,300,400,500,600,700,800,900,1000,...,9000,9100,9200,9300,9400,9500,9600,9700,9800,9900
d_so,,,,,,,,,,,,,,,,,,,,,
99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [317]:
# remove these
f = 3
output = o_list[8]


def get_magnification(row, d_so):
    d_sd = row.name
    return (d_sd/d_so)
    
ni = 50 # number of options in each d_sd and d_so
# d_so_s = pd.Series(np.linspace(d_so_min,d_so_max, ni), name='d_so')
d_so_s = pd.Series(np.arange(50,650, 50), name='d_so')

ni = 50 # number of options in each d_sd and d_so
# d_sd_s = pd.Series(np.linspace(d_sd_min,d_sd_max, ni), name='d_sd')
d_sd_s = pd.Series(np.arange(50,650, 50), name='d_sd')
df_distances = pd.DataFrame(index=d_so_s, columns=d_sd_s) # rows: source to object; cols: source to detector distance

M = df_distances.apply(lambda x: get_magnification(x, df_distances.index), axis=0) # dataframe with all the magnifications
B_do = M.copy().rdiv(B_d) # blur in detector scaled to object plane
B_fo = (1-(M.copy().rdiv(1))).multiply(f)# blur in focal spot scaled to object plane
B_o = np.sqrt(B_do.multiply(B_do) + B_fo.multiply(B_fo))# overall resolution (gaussian blurs B_do and B_fo assumed)

# flux is not the same!
flux_s = (output/4/np.pi/d_sd_s**2) * output_bias # flux at the detector
flux_s.name = 'flux_at_detector'
flux_utilization = M**2  # flux utilization?

imaging_time_raw_s = detector_counts/flux_s/detector_efficiency
imaging_time_raw_s.name ='raw_imaging_time'

def calc_effective_time(row, imaging_time_raw_s):
    # multiply each element in the row with the respective element in imaging_time_raw_s
#     print(row)

    return row * imaging_time_raw_s.values

df_imaging_time_effective = flux_utilization.apply(lambda x: calc_effective_time(x, imaging_time_raw_s), axis=1)

# zero out forbidden values
# object to detector distance
def get_d_od(row, d_so):
    d_sd = row.name
    return (d_sd-d_so)

d_od = df_distances.apply(lambda x: get_d_od(x, df_distances.index), axis=0) # dataframe with all the magnifications
d_od_forbidden = d_od<d_od_min
M_max = det_size_max/FOV_min # maximum allowed magnification
M_forbidden = M>M_max
B_o_forbidden = B_o>resolution_goal

dfs = [B_o, flux_utilization, df_imaging_time_effective]
for df in dfs:
    df[B_o_forbidden] = 0
    df[M_forbidden] = 0
    df[d_od_forbidden] = 0

# set the zeros to ~inf to not mess up finding minimum
df_imaging_time_effective[df_imaging_time_effective==0] = 1e10;
best_imaging_time = np.min(np.min(df_imaging_time_effective))
best_d_sd = df_imaging_time_effective.idxmin().idxmin()
best_d_so =  df_imaging_time_effective.loc[:,best_d_sd].idxmin()

In [318]:
df_imaging_time_effective[df_imaging_time_effective == best_imaging_time]

d_sd,50,100,150,200,250,300,350,400,450,500,550,600
d_so,,,,,,,,,,,,
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,NaN,NaN,NaN,NaN,NaN,952.172737,NaN,NaN,NaN,NaN,NaN,NaN
250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [319]:
best_d_so, best_d_sd

(50, 50)

In [320]:
best_imaging_time

952.1727366341005

In [229]:
df = pd.DataFrame({'angles': [1, 2, 3, 4],'degrees': [55, 66, 44, 12]})
a = df.multiply(3)
a

,angles,degrees
0,3,165
1,6,198
2,9,132
3,12,36


In [230]:
b = df.rdiv(1)
b

,angles,degrees
0,1.000000,0.018182
1,0.500000,0.015152
2,0.333333,0.022727
3,0.250000,0.083333


In [231]:
df

,angles,degrees
0,1,55
1,2,66
2,3,44
3,4,12


In [114]:
0.272727+0.018595

0.29132199999999997